### 문제 1-1 : 기본 체인 만들기 - AI 요리사

In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    """
    당신은 창의적인 AI 요리사입니다.
    사용자가 입력한 재료로 만들 수 있는 요리 하나를 추천하고,
    간단한 레시피를 3단계로 설명해주세요.

    사용자가 입력한 재료는 다음과 같습니다: {ingredients}
    
    출력 형식은 아래와 같아야 합니다:

    {ingredients}로 만들 수 있는 요리를 추천드립니다!
    
    추천 요리: <요리 이름>
    재료: <입력 재료>
    조리법:
    1. ...
    2. ...
    3. ...
    """)
prompt

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
# output parser 지정
output_parser = StrOutputParser()
# chain 연결 (LCEL)
chain = prompt | llm | output_parser

# chain 호출
try:
    result = chain.invoke({"ingredients": "계란, 밥, 김치"})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

 계란, 밥, 김치로 만들 수 있는 요리를 추천드립니다!

 추천 요리: 김치 볶음밥
 재료: 계란, 밥, 김치
 조리법:
1. 김치를 잘게 썰고, 계란은 풀어서 준비합니다.
2. 팬에 기름을 두르고 썰어 놓은 김치를 볶다가 밥을 넣어 함께 볶습니다.
3. 계란을 넣고 잘 섞어준 뒤, 간을 맞추면 김치 볶음밥이 완성됩니다!


### 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import re

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
# output parser 지정
output_parser = StrOutputParser()

# Step 1: 장르를 입력받아 영화 1편 추천
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 추천 전문가입니다."),
    ("human", "{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요. 영화 제목은 **이중 별표**로 감싸주세요.")
])

# Step 2: 추천받은 영화의 3줄 줄거리 요약 제공
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다."),
    ("human", 
    """
    {movie}라는 영화에 대해 아래 형식처럼 요약해 주세요:

    줄거리:
    1. (첫 번째 문장)
    2. (두 번째 문장)
    3. (세 번째 문장)

    위 형식에 따라 문장을 나눠서 간결하게 작성해 주세요.
    """)
])

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | output_parser

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = prompt2 | llm | output_parser

try:
    genre_input = "액션"
    step1_output = chain1.invoke({"genre": genre_input}).strip()
    print(f'입력: "{genre_input}"')

    # 제목만 추출
    match = re.search(r"\*\*(.+?)\*\*", step1_output)
    if match:
        movie_title = match.group(1).strip()
    else:
        movie_title = step1_output.split("\n")[0].replace("추천 영화:", "").strip()

    print(f'1단계 결과: "{movie_title}" 추천')

    step2_output = chain2.invoke({"movie": movie_title}).strip()
    print("2단계 결과:")
    print(f"영화: {movie_title}")
    print(step2_output)

except Exception as e:
    print(f"오류 발생: {e}")

입력: "액션"
1단계 결과: "존 윅" 추천
2단계 결과:
영화: 존 윅
줄거리:

1. 전직 히트맨 존 윅은 은퇴 후 평화로운 삶을 살고자 하지만, 그의 집에 침입한 한 젊은 범죄자가 존 윅의 차를 훔치고, 그의 개를 죽이는 사건이 발생합니다.
2. 이 사건으로 인해 존 윅은 다시 한번 전직 히트맨으로서의 삶을 살게 되며, 범죄자의 보스인 비고르 타라소프에게 복수를 결심합니다.
3. 존 윅은 타라소프의 조직과 싸우며, 그 과정에서 많은 총격과 액션 장면이 펼쳐지게 되며, 존 윅은 과연 타라소프의 조직을 무너뜨리고 복수에 성공할 수 있을지 관심이 집중됩니다.


### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [15]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
# output parser 지정
output_parser = StrOutputParser()

# 1. 예시 프롬프트
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "최근 연구에 따르면 고양이는 사람의 감정을 인식할 수 있으며, 주인의 스트레스 수준에 따라 행동이 달라지는 것으로 나타났다. 전문가들은 고양이와의 교감이 정신 건강에 긍정적인 영향을 줄 수 있다고 설명했다.",
        "keywords": "고양이, 감정인식, 반려동물"
    }
]

# 2. Few-Shot 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 3. 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

chain = final_prompt | llm

result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

키워드: 제미나이, AI, 구글
